In [3]:
#LOIS NOTEBOOK
import pandas as pd

In [4]:
#datasets from ml-25m
movies_df = pd.read_csv('./datasets/ml-25m/movies.csv')
ratings_df = pd.read_csv('./datasets/ml-25m/ratings.csv')
links_df = pd.read_csv('./datasets/ml-25m/links.csv')
tags_df = pd.read_csv('./datasets/ml-25m/tags.csv')
genome_scores_df = pd.read_csv('./datasets/ml-25m/genome-scores.csv')
genome_tags_df = pd.read_csv('./datasets/ml-25m/genome-tags.csv')

In [5]:
#creation of main table for machine learning
merged_df = pd.merge(movies_df, ratings_df, on='movieId', how='left')
tags_df['tag'] = tags_df['tag'].astype(str)
tags_grouped = tags_df.groupby('movieId')['tag'].apply(lambda x: ' '.join(x)).reset_index()
merged_df = pd.merge(merged_df, tags_grouped, on='movieId', how='left')

In [10]:
final_movies_df = merged_df[['movieId', 'genres', 'userId', 'rating', 'tag']]
final_movies_df = final_movies_df.rename(columns={'movieId': 'movie_id', 'userId': 'user_id'})
data = final_movies_df

,movie_id,genres,user_id,rating,tag
0,1,Adventure|Animation|Children|Comedy|Fantasy,2.0,3.5,Owned imdb top 250 Pixar Pixar time travel chi...
1,1,Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,Owned imdb top 250 Pixar Pixar time travel chi...
2,1,Adventure|Animation|Children|Comedy|Fantasy,4.0,3.0,Owned imdb top 250 Pixar Pixar time travel chi...
3,1,Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,Owned imdb top 250 Pixar Pixar time travel chi...
4,1,Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,Owned imdb top 250 Pixar Pixar time travel chi...
...,...,...,...,...,...
25003466,209157,Drama,119571.0,1.5,NaN
25003467,209159,Documentary,115835.0,3.0,NaN
25003468,209163,Comedy|Drama,6964.0,4.5,NaN
25003469,209169,(no genres listed),119571.0,3.0,NaN


In [22]:
! wget https://raw.githubusercontent.com/shenweichen/DeepMatch/master/examples/preprocess.py -O preprocess.py

--2024-03-31 19:48:21--  https://raw.githubusercontent.com/shenweichen/DeepMatch/master/examples/preprocess.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8001::154, 2606:50c0:8000::154, 2606:50c0:8003::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6705 (6.5K) [text/plain]
Saving to: ‘preprocess.py’

preprocess.py       100%[===================>]   6.55K  1.68KB/s    in 3.9s    

2024-03-31 19:48:48 (1.68 KB/s) - ‘preprocess.py’ saved [6705/6705]



In [37]:
import tensorflow as tf
from deepctr.feature_column import SparseFeat, VarLenSparseFeat
from deepmatch.models import *
from deepmatch.utils import sampledsoftmaxloss, NegativeSampler
# from preprocess import gen_data_set, gen_model_input
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model

pad_sequences = tf.keras.utils.pad_sequences

# if __name__ == "__main__":

#     data = final_movies_df
#     data['genres'] = list(map(lambda x: x.split('|')[0], data['genres'].values))

#     sparse_features = ["movie_id", "user_id", "genres"]
#     SEQ_LEN = 50

#     # 1.Label Encoding for sparse features,and process sequence features with `gen_date_set` and `gen_model_input`

#     feature_max_idx = {}
#     for feature in sparse_features:
#         lbe = LabelEncoder()
#         data[feature] = lbe.fit_transform(data[feature]) + 1
#         feature_max_idx[feature] = data[feature].max() + 1

#     user_profile = data[["user_id"]].drop_duplicates('user_id')

#     item_profile = data[["movie_id"]].drop_duplicates('movie_id')

#     user_profile.set_index("user_id", inplace=True)

#     user_item_list = data.groupby("user_id")['movie_id'].apply(list)

#     train_set, test_set = gen_data_set(data, SEQ_LEN, 0)

#     train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)
#     test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)

#     # 2.count #unique features for each sparse field and generate feature config for sequence feature

#     embedding_dim = 16

#     user_feature_columns = [SparseFeat('user_id', feature_max_idx['user_id'], embedding_dim),
#                             VarLenSparseFeat(SparseFeat('hist_movie_id', feature_max_idx['movie_id'], embedding_dim,
#                                                         embedding_name="movie_id"), SEQ_LEN, 'mean', 'hist_len'),
#                             VarLenSparseFeat(SparseFeat('hist_genres', feature_max_idx['genres'], embedding_dim,
#                                                         embedding_name="genres"), SEQ_LEN, 'mean', 'hist_len')
#                             ]

#     item_feature_columns = [SparseFeat('movie_id', feature_max_idx['movie_id'], embedding_dim)]

#     from collections import Counter

#     train_counter = Counter(train_model_input['movie_id'])
#     item_count = [train_counter.get(i, 0) for i in range(item_feature_columns[0].vocabulary_size)]
#     sampler_config = NegativeSampler('frequency', num_sampled=5, item_name='movie_id', item_count=item_count)

#     # 3.Define Model and train

#     import tensorflow as tf

#     if tf.__version__ >= '2.0.0':
#         tf.compat.v1.disable_eager_execution()
#     else:
#         K.set_learning_phase(True)

#     model = YoutubeDNN(user_feature_columns, item_feature_columns, user_dnn_hidden_units=(64, embedding_dim),
#                        sampler_config=sampler_config)
#     # model = MIND(user_feature_columns, item_feature_columns, dynamic_k=False, k_max=2,
#     #              user_dnn_hidden_units=(64, embedding_dim), sampler_config=sampler_config)

#     model.compile(optimizer="adam", loss=sampledsoftmaxloss)

#     history = model.fit(train_model_input, train_label,  # train_label,
#                         batch_size=256, epochs=1, verbose=1, validation_split=0.0, )

#     # 4. Generate user features for testing and full item features for retrieval
#     test_user_model_input = test_model_input
#     all_item_model_input = {"movie_id": item_profile['movie_id'].values}

#     user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
#     item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)

#     user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
#     # user_embs = user_embs[:, i, :]  # i in [0,k_max) if MIND
#     item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

#     print(user_embs.shape)
#     print(item_embs.shape)